## 面试爬虫任务：抓取有道翻译接口http://fanyi.youdao.com/ ，完成中文翻英文

### 一. 需求分析
        1. 每一个爬虫，无外乎两种需求：
            a. 抓定向数据
            b. 模拟操作，比如常见的自动抢购、投简历等
           题目属于前者，最终目的是完成中文到英文的翻译
           
        2. 老套路，用chrome打开 http://fanyi.youdao.com/ 并切换到开发者模式，查看网络，随便查几个词测试一下并截图分析如下：

![](http://peteryang.club./youdao/page_analysis.jpg)

        3. 从上面截图可以看到快速得出如下信息：
            a. 查词使用AJAX异步请求，请求方式是POST一个表单，返回数据是json格式
            b. 多次对比发现表单数据只有三个是会变的，i是所查词语，salt的值经过如下在线时间戳转换工具确认是linux时间戳，sign 应该是salt加上某些参数的hash值。

使用的链接：1. [时间戳转换](http://tool.lu/timestamp)    2.  [JavaScript 代码格式化](http://tool.oschina.net/codeformat/js/)

        4. 经过上述初步分析，这个站点是动态网站，有反爬措施，破题的关键在于找到 *sign* 这个参数是如何生成的，既然是hash值，那肯定是不会像知乎登陆的时候需要的 _xsrf 一样放在页面，一查果然没有，并且观察是每查一个词只有一个请求，那也应该不是在服务器上生成的，只能是在页面js脚本动态生成的，点击查看源代码，还好只有两个脚本，第一个不太像，点击fanyi js脚本，chrome乱码，找个在线格式化网站格式化一下，行数7000+，copy到notepad++，查找salt和sign关键字，还好只有三个结果，找对应上面form data中version为2.1的截图如下:

![js 脚本](http://peteryang.club./youdao/js.jpg)

![](http://peteryang.club./youdao/sign_create.jpg)

        从上面我们可以看到，sign = md5(client + i + salt +A ) 生成得到的，前三个参数不难得到，第四个 *A* 是大写，按照大多数程序语言的默认设定这应该是个常量，再查找A，全词匹配，果然查到了：

![](http://peteryang.club./youdao/A.jpg)

      5. 分析到这里基本就很清楚了，编写这个爬虫难点在于生成这个sign参数，搜一下Python如何生成hash, 呦，挺好，还是标准库实现的：

![](http://peteryang.club./youdao/hash.jpg)

### 二. 开发环境
        1. python3.5 ；
        2. requests   # 用作http请求 
        3. hashlib   # 生成请求参数sign
        4. jupyter notebook  # 提供python运行环境的同时用于撰写Markdowm文档，非常好的工具

### 三. 代码编写

In [64]:
import requests
import hashlib
import random
import time

def get_data(word):
    salt = str(int(time.time()*1000) + random.randint(1,10))
    A = "rY0D^0'nM0}g5Mm1z%1G4"
    sign = hashlib.md5(("fanyideskweb" + word + salt + A).encode('utf8')).hexdigest()   # Create hash sign
    
    url = 'http://fanyi.youdao.com/translate_o?smartresult=dict&smartresult=rule'
    
    data = {}
    data['i'] = word
    data['salt'] = salt
    data['sign'] = sign
    data['from'] = 'AUTO'
    data['to'] = 'AUTO'
    data['smartresult'] = 'dict'
    data['client'] = 'fanyideskweb'
    data['doctype'] = 'json'
    data['version'] = '2.1'
    data['keyfrom'] = 'fanyi.web'
    data['action'] = 'FY_BY_REALTIME'
    data['typoResult'] = 'true'

    headers = {
         'Accept':'application/json, text/javascript, */*; q=0.01',
         'Accept-Encoding':'gzip, deflate',
         'Accept-Language':'zh-CN,zh;q=0.8,en;q=0.6',
         'Connection':'keep-alive',
         'Content-Length':'205',
         'Content-Type':'application/x-www-form-urlencoded; charset=UTF-8',
         'Cookie':'OUTFOX_SEARCH_USER_ID=-1952769096@123.58.182.243; OUTFOX_SEARCH_USER_ID_NCOO=299132983.56201506; _ntes_nnid=2be06273f9a84c503694b14e705dc9d4,1503010352270; JSESSIONID=aaaOR3Z_jEmnuc7ysuQ8v; UM_distinctid=15f2c092692501-0bc916153c00d5-1227170b-13c680-15f2c09269354a; NTES_SESS=MV6x1csO53ep_nKOlpk_yAn.Yg.1fkWwAntD7bKMMPGBbX4tbOxW2SkdxGmK6Xu_.IeBeM87MpQplZBbZL.Zy3xK5YmsDdjgqdobLzpn2hra0a5PnssQkR6S4r5eI.1ln19IA0nLHPh6U3fCvgYnPa5YVImypa0q9WrnsmVgVcHjyb_FVGBXWnFiR0I8wmk4E; S_INFO=1508243941|0|3&100##|m15917916221_1; P_INFO=m15917916221_1@163.com|1508243941|0|search|00&99|gud&1508235466&search#gud&440100#10#0#0|159221&1|search&mail163|15917916221@163.com; ___rl__test__cookies=1508281766130',
         'Host':'fanyi.youdao.com',
         'Origin':'http://fanyi.youdao.com',
         'Referer':'http://fanyi.youdao.com/',
         'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
         'X-Requested-With':'XMLHttpRequest',
         }
    try:
        res = requests.post(url = url, data = data, headers = headers, timeout = 8)
        if res.status_code == 200:
            return res.json()
    except Exception as e:
        print("Raise Exception :",e)

def parse_json(json):
    if isinstance(json,dict):
        print(json)
        print()
        source = []
        target = []
        for d in json["translateResult"][0]:
            source.append(d['src'])
            target.append(d['tgt'])
        print('原文： ',' '.join(source))
        print()
        print('译文： ', ' '.join(target))
        
    
if __name__ == "__main__":
    word = "Python下有许多款不同的 Web 框架。Django是重量级选手中最有代表性的一位。许多成功的网站和APP都基于Django"
    json = get_data(word)
    if json is not None:
        parse_json(get_data(word))

{'type': 'zh-CHS2en', 'translateResult': [[{'src': 'Python下有许多款不同的 Web 框架。', 'tgt': 'There are many different Web frameworks under Python.'}, {'src': 'Django是重量级选手中最有代表性的一位。', 'tgt': 'Django is the most representative of the heavyweights.'}, {'src': '许多成功的网站和APP都基于Django', 'tgt': 'Many successful websites and apps are based on Django'}]], 'errorCode': 0}

原文：  Python下有许多款不同的 Web 框架。 Django是重量级选手中最有代表性的一位。 许多成功的网站和APP都基于Django

译文：  There are many different Web frameworks under Python. Django is the most representative of the heavyweights. Many successful websites and apps are based on Django


### 四. 总结

#### 1. 为什么要爬虫？
    因为懒。懒是人类生产力进步的强大动力，做一年多运维深谙此理，每一个任务，如果重复率超过3次，都尽量用自动化来实现，爬虫也是如此，前两天投简历的时候因为懒得挨个投，干脆用python 的selenium库写了个爬虫脚本去自动投递。
    
    在信息化大数据时代，掌握数据结构化抓取是几乎已经成为干IT的必备技能。

#### 2. 如何对目标站点进行分析？
    爬取目标分析这点我认为是写爬虫整个过程中最重要的，以下总结以下我常用的手段：
    
     a. 目标站点架构分析：ping一下目标站点域名获取ip，把ip放到这个神奇的网站搜一下：[shodan 神一样的网站](https://www.shodan.io/host/) 用谁家的云开放的什么端口都一目了然，可以初步对该站点做一些判断了，比如用的什么缓存，用什么数据库，用的什么web服务器，有时候这些信息可能是突破点       

![](http://peteryang.club./youdao/shoda.jpg)
  

    b. 接下来用浏览器正常浏览一下目标站点，切到开发者模式，注意看请求地址，请求头，请求数据都是什么，哪些数据是常量哪些是变量，有AJAX请求看一下参数都有些什么规律，再找一下目标站点的根节点（根节点是指能去到所有链接的节点），比如知乎根节点可以是话题广场--》所有话题--》所有问题--》回答用户...网易云音乐--》所有歌手--》所有音乐--》所有评论/歌单...一般是这么个套路
  
    c. 如果目标站点有移动端APP或者开放的API，用Fiddler或者Charles抓包看看请求接口和参数，优先爬移动端接口，因为一般反爬程度要比网站轻，同时返回的数据一般是 XML 或者 JSON，比较干净而且省带宽，关键是能拼出正确的请求参数。
  
    d. 清楚自己要什么数据，选用最省成本的方式抓取（比如时间成本，技术成本），一般网站用 requests + bs4 + lxml 都能拿下了，我比较常用组合是requests + lxml，主要是xpath用起来比bs4爽，而且可以少一个包依赖，对于像QQ空间或京东淘宝这种业务逻辑复杂，大量数据都是js脚本动态生成的网站，用selenium + chrome/phantomjs 就没那么糟心，不用挨个拼参数，直接模仿人的操作。
    
    像题目要求如果仅仅是翻译的话找个API爬会方便很多,下面是我在linux上常用的词典工具：

In [61]:
'''这是一个爬取爱词霸的api的字典爬虫，可以再linux下做命令行的字典用'''
import requests
import sys

word = ('消失')
# word = sys.argv[1]
res = requests.get('http://dict-co.iciba.com/api/dictionary.php?key=100113A76C05A2D043654247C29BDF50&type=json&w='+ word.lower())
print(word)
for d in (res.json()['symbols'][0]['parts'][0]['means']):
    print(d['word_mean'])

消失
disappear
vanish
dissolve
die away
clear away 


#### 3. 有哪些常见的反反爬手段
    a. 请求头，把浏览器中出现的请求头能带的都带上，尽量模拟人的访问
    
    b. User-Agent, 随机变换UA
    
    c. IP限制，随机换代理IP，有钱直接买，没钱搜一下免费的，公司级别的爬虫一般会有多台服务器做分布式来爬
    
    d. 在页面中用js动态生成请求参数，像本题中的hash sign，两种方法，一是查看相应的js文件看一下参数是如何构造的，第二是直接用webdriver渲染获得该参数，再用requests把参数带上访问站点

#### 4. 感悟

    a. 爬虫只是手段，数据才是目的，用什么手段看具体要求，比如我投简历的爬虫为什么用selenium而不是requests的原因是懒得去拼参数，投个简历对速度没啥要求，selenium模拟人的程度更高，花五个小时跑半个钟跟花半个钟跑五个小时看自己具体需求。
   
    b. 一个原则是不要对目标站点做高频访问请求，爬人家数据的同时不给人添麻烦，所有程序写的时候注意sleep一下，最好是自适应的。可以配合数据库或者队列做增量爬取。
   
    c. 大规模结构化爬取尽量用scrapy等框架，多线程多进程用 multiprocessing 和其dummy库，好处是库的接口一样，可以随时切换。异步协程库gevent对爬虫这种IO阻塞的任务是非常不错的选择，速度有几十到上百倍提升，听说python3.5的asyncio挺好，毕竟官方实现，未来的趋势，有空要学一下。
    
    d. 总的来说我认为爬虫其实还挺好玩的，很多时候碰到不懂的就要大胆去猜，然后去验证，有种狄仁杰探案的感觉。。。题目中的 *salt* 参数猜测是 linux时间戳是因为之前对搜房网APP抓包的时候发现也有过这个参数，而 *sign* 参数让我想起了好几年前在网上流传的数百万小米论坛数据泄露，当时还花钱买了一份，后来发现密码段存的是 hash值，还是加了盐的，啥用出也没有，所有对这个影响挺深。
    
    e. 善用搜索。

#### 以下是几个我写的小项目：

抓了58万网易云音乐的歌曲id和歌名，下一步打算抓歌曲id对应的歌单和歌手，做个app给自己用，目前未最终完成：
[Github - NetEase music spider](https://github.com/ananan/python-code/blob/master/another_songs.py)

用python cgi做了个界面，有点粗糙，后续打算用django做个完整点的，学一下bootstrap和jQuery，项目链接如下：
[Music Box Web](http://peteryang.club./cgi-bin/songs)

在前程无忧上自动投简历： 
[find_job_in_51.py](https://github.com/ananan/python-code/blob/master/find_job_in_51job.py)

##### 非常感谢看完这篇文章，一个题目零零碎碎写成了一个博客，文中若有疏漏欢迎指正，我的邮箱：15917916221@163.com